<a href="https://colab.research.google.com/github/mfadlisy/data-portfolio/blob/main/optimizing-bike-sharing-strategic-marketing/cleaning-data-process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [2]:
# Import library
import numpy as np
import pandas as pd
from geopy.distance import geodesic

In [3]:
# Load data
data = pd.read_csv('/content/drive/MyDrive/Learn Data/Project Case Study/Optimizing Rental Bike Campaigns/dataset/cyclistic.csv')

In [4]:
# Show the top 5
data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A50255C1E17942AB,classic_bike,2022-10-14 17:13:30,2022-10-14 17:19:39,Noble St & Milwaukee Ave,13290,Larrabee St & Division St,KA1504000079,41.900680,-87.662600,41.903486,-87.643353,member
1,DB692A70BD2DD4E3,electric_bike,2022-10-01 16:29:26,2022-10-01 16:49:06,Damen Ave & Charleston St,13288,Damen Ave & Cullerton St,13089,41.920037,-87.677937,41.854967,-87.675700,casual
2,3C02727AAF60F873,electric_bike,2022-10-19 18:55:40,2022-10-19 19:03:30,Hoyne Ave & Balmoral Ave,655,Western Ave & Leland Ave,TA1307000140,41.979879,-87.681902,41.966400,-87.688704,member
3,47E653FDC2D99236,electric_bike,2022-10-31 07:52:36,2022-10-31 07:58:49,Rush St & Cedar St,KA1504000133,Orleans St & Chestnut St (NEXT Apts),620,41.902274,-87.627692,41.898203,-87.637536,member
4,8B5407BE535159BF,classic_bike,2022-10-13 18:41:03,2022-10-13 19:26:18,900 W Harrison St,13028,Adler Planetarium,13431,41.874754,-87.649807,41.866095,-87.607267,casual


In [5]:
# show shape data
data.shape

(5674399, 13)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5674399 entries, 0 to 5674398
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 562.8+ MB


# Handling duplicate data and missing value

In [7]:
data.duplicated().sum()

0

In [8]:
data.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    873186
start_station_id      873318
end_station_name      926160
end_station_id        926301
start_lat                  0
start_lng                  0
end_lat                 6642
end_lng                 6642
member_casual              0
dtype: int64

In [9]:
missing_percentage = (data.isnull().sum() / len(data)) * 100
print(missing_percentage)

ride_id                0.000000
rideable_type          0.000000
started_at             0.000000
ended_at               0.000000
start_station_name    15.388167
start_station_id      15.390493
end_station_name      16.321729
end_station_id        16.324213
start_lat              0.000000
start_lng              0.000000
end_lat                0.117052
end_lng                0.117052
member_casual          0.000000
dtype: float64


In [10]:
data = data.fillna(0)

In [11]:
data.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

# Change datatype

In [12]:
# Change datatype for 'started_at' column
data['started_at'] = pd.to_datetime(data['started_at'])

# Change datatype for 'ended_at' column
data['ended_at'] = pd.to_datetime(data['ended_at'])

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5674399 entries, 0 to 5674398
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 562.8+ MB


# Create new columns

In [14]:
# Create 'day_name' column
data['day_name'] = data['started_at'].dt.day_name()

In [15]:
# Create 'duration' column
data['duration_minutes'] = (data['ended_at'] - data['started_at']).dt.total_seconds() / 60
data['duration_minutes'] = data['duration_minutes'].astype(int)

In [16]:
# Create 'is_weekend' column
data['is_weekend'] = data['day_name'].apply(lambda x: 'Yes' if x in ['Saturday', 'Sunday'] else 'No')

In [18]:
# Create 'distance' column
def distance(data):
    start_coords = (data['start_lat'], data['start_lng'])
    end_coords = (data['end_lat'], data['end_lng'])
    return geodesic(start_coords, end_coords).kilometers

data['distance'] = data.apply(distance, axis=1)

In [21]:
data.sample(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,day_name,duration_minutes,is_weekend,distance
1388347,7151C9B0F5F75B42,electric_bike,2023-02-27 16:53:42,2023-02-27 17:02:26,Ashland Ave & Division St,13061,0,0,41.903333,-87.667809,41.920000,-87.650000,member,Monday,8,No,2.368607
2183635,D400E2F6C8239152,classic_bike,2023-05-27 18:02:42,2023-05-27 18:06:46,Southport Ave & Belmont Ave,13229,Ashland Ave & Wellington Ave,13269,41.939478,-87.663748,41.936083,-87.669807,member,Saturday,4,Yes,0.628244
114717,D5A21A7AB1093051,electric_bike,2022-10-07 15:53:04,2022-10-07 15:56:26,0,0,Ellis Ave & 60th St,KA1503000014,41.790000,-87.600000,41.785097,-87.601073,member,Friday,3,No,0.551809
1130702,48C155EAA85E359C,classic_bike,2023-01-24 18:12:22,2023-01-24 18:19:55,Wabash Ave & Roosevelt Rd,TA1305000002,Calumet Ave & 21st St,15546,41.867227,-87.625961,41.854184,-87.619154,member,Tuesday,7,No,1.555024
1882678,9BB2B7AED3162A6C,classic_bike,2023-04-08 18:07:18,2023-04-08 18:32:54,Albany Ave & Bloomingdale Ave,15655,Albany Ave & Bloomingdale Ave,15655,41.914027,-87.705126,41.914027,-87.705126,casual,Saturday,25,Yes,0.000000


In [25]:
data.to_csv('/content/drive/MyDrive/Learn Data/Project Case Study/Optimizing Rental Bike Campaigns/dataset/cyclistic_cleaned.csv', index=False)